# Set up Julia processes and output dataframe

In [ ]:
using Distributed
addprocs(7)

In [ ]:
@everywhere begin
    using Catalyst, DifferentialEquations, Distributions, CairoMakie
    using DataFrames
    using Parquet2
end

In [ ]:
# Make the output directory if it does not exist
outdir = "$(@__DIR__)/../../output/modeling/julia_missplicing_simulations"
mkpath(outdir)

# Make output dataframe
data = DataFrame(copynum=Int[], reg_gene=Int[], unreg_gene=Int[], design=String[], moi=Float64[], risc=Int[])

# Define reaction networks

Constituent reaction networks

In [ ]:
@everywhere begin
    
    # RISC-mediated knockdown of the regulated gene, `regulated_mRNA`
    dicer_risc_rn = @reaction_network dicer_RISC_complex begin
        @parameters begin
            r_dicer=1e-3            # pre-miRNA processing rate constant [1/s]
            δ_mi=2.88e-4            # miRNA degradation rate constant [1/s]
            k_miRNA_bind=1e-5       # miRNA-RISC binding rate constant [1/mol*s]
            k_miRNA_deg=2.16e-5  # miRNA-RISC unbinding rate constant [1/s]
            k_mRNA_bind=1.84e-6     # RISC complex formation rate constant[1/mol*s]
            k_mRNA_unbind=0.303     # RISC complex deformation rate constant [1/s]
            k_deg=7e-3              # RISC-mediated mRNA degradation rate constant [1/s]
        end
        # dicer
        r_dicer, pre_miRNA --> miRNA
        # degradation
        δ_mi, miRNA --> ∅
        # Knockdown
        k_miRNA_bind, risc + miRNA --> risc_miRNA
        k_miRNA_deg, risc_miRNA --> risc
        (k_mRNA_bind, k_mRNA_unbind), risc_miRNA + regulated_mRNA <--> risc_miRNA_mRNA
        k_deg, risc_miRNA_mRNA --> risc_miRNA
    end

    # RISC-mediated knockdown of the immature unspliced transcript, 'immature_mRNA'
    immature_risc_rn = @reaction_network immature_RISC_complex begin
        @parameters begin
            k_mRNA_bind=1.84e-6     # RISC complex formation rate constant[1/mol*s]
            k_mRNA_unbind=0.303     # RISC complex deformation rate constant [1/s]
            k_deg=7e-3              # RISC-mediated mRNA degradation rate constant [1/s]
        end
        # Knockdown
        (k_mRNA_bind, k_mRNA_unbind), risc_miRNA + immature_mRNA <--> risc_miRNA_im_mRNA
        k_deg, risc_miRNA_im_mRNA --> risc_miRNA
    end

    # Regulated gene dynamics
    regulated_gene_rn = @reaction_network regulated_gene_dynamics begin
        @parameters begin
            δ_m=2.88e-4 # Mature mRNA degradation rate constant [1/s]
            δ_p=9.67e-5 # Protein degradation rate constant [1/s] 
            α_p=3.33e-4 # Translation rate constant [1/s]
            ζ=0.0       # Translation efficiency for RISC complex [unitless], Range: 0 to 1
        end
        # Production of protein
        α_p, regulated_mRNA --> protein + regulated_mRNA
        α_p, control_mRNA --> control_protein + control_mRNA
        α_p * ζ, risc_miRNA_mRNA --> protein + risc_miRNA_mRNA
        # Degradation of products
        δ_m, regulated_mRNA --> ∅
        δ_m, control_mRNA --> ∅
        δ_p, protein --> ∅
        δ_p, control_protein --> ∅
    end

    # Immature mRNA translation dynamics
    immature_translation_correct_rn = @reaction_network immature_correct_translation begin
        @parameters begin
            δ_m=2.88e-4 # Mature mRNA degradation rate constant [1/s]
            δ_p=9.67e-5 # Protein degradation rate constant [1/s] 
            α_p=3.33e-4 # Translation rate constant [1/s]
            ζ=0.0       # Translation efficiency for RISC complex [unitless], Range: 0 to 1
        end
        # Production of protein
        α_p, immature_mRNA --> protein + immature_mRNA
        α_p * ζ, risc_miRNA_im_mRNA --> protein + risc_miRNA_im_mRNA
    end
    immature_translation_incorrect_rn = @reaction_network immature_incorrect_translation begin
        @parameters begin
            δ_m=2.88e-4 # Mature mRNA degradation rate constant [1/s]
            δ_p=9.67e-5 # Protein degradation rate constant [1/s] 
            α_p=3.33e-4 # Translation rate constant [1/s]
            ζ=0.0       # Translation efficiency for RISC complex [unitless], Range: 0 to 1
        end
        # Production of protein
        α_p, immature_mRNA --> immature_mRNA + incorrect_protein
        α_p * ζ, risc_miRNA_im_mRNA --> incorrect_protein + risc_miRNA_im_mRNA
        # Degradation of products
        δ_p, incorrect_protein --> ∅
    end

    # Production of a regulated transcript with its own miRNA intron
    intronic_regulated_gene_rn = @reaction_network intronic_regulated_expression begin
        @parameters begin
            α_im=4.67e-2        # Transcription rate constant [1/s]
            δ_im=2.88e-4        # Immature mRNA degradation rate constant [1/s]
            r_splicing=2.0e-3   # Splicing rate constant [1/s]
            r_drosha=1e-2       # pri-miRNA processing rate constant [1/s]
        end
        # Transcription
        α_im, gene --> immature_mRNA + gene
        α_im, gene --> control_mRNA + gene
        # Degradation of products
        δ_im, immature_mRNA --> ∅  
        # Splicing
        r_splicing, immature_mRNA --> regulated_mRNA + pri_miRNA
        # miRNA processing: drosha
        r_drosha, pri_miRNA --> pre_miRNA
    end

    # Production of an regulated transcript with its own miRNA intron
    missplicing_rn = @reaction_network missplicing begin
        @parameters begin
            r_mRNA_only=0.0   # Splicing rate constant [1/s]
            r_miR_only=0.0   # Splicing rate constant [1/s]
        end
        # Splicing
        r_mRNA_only, immature_mRNA --> regulated_mRNA
        r_miR_only, immature_mRNA --> pri_miRNA
    end

    # Production of a regulated gene transcript, without an internal intron
    nonintronic_regulated_gene_rn = @reaction_network gene_ts_expression begin
        @parameters begin
            α_m=4.67e-2     # Transcription rate constant [1/s]
        end
        α_m, gene --> regulated_mRNA + gene
        α_m, gene --> control_mRNA + gene
    end

    # Production of an unregulated transcript with the miRNA intron
    intronic_unregulated_gene_rn = @reaction_network intronic_unregulated_expression begin
        @parameters begin
            α_im=4.67e-2       # Transcription rate constant [1/s]
            α_p=3.33e-4        # Translation rate constant [1/s]
            δ_m=2.88e-4        # Mature mRNA degradation rate constant [1/s]
            δ_p=9.67e-5        # Protein degradation rate constant [1/s] 
            δ_im=2.88e-4       # Immature mRNA degradation rate constant gene 2 [1/s]
            r_splicing=2.0e-3   # Splicing rate constant [1/s]
            r_drosha=1e-2       # pri-miRNA processing rate constant [1/s]
        end
        # Transcription
        α_im, miRNA_gene --> immature_mRNA + miRNA_gene
        # Degradation of products
        δ_im, immature_mRNA --> ∅
        # Splicing
        r_splicing, immature_mRNA --> pri_miRNA
        # miRNA processing: drosha
        r_drosha, pri_miRNA --> pre_miRNA
    end

end

Overall design reaction networks

In [ ]:
@everywhere begin
    common_rn = extend(dicer_risc_rn, regulated_gene_rn)
    CDS_intron_rn = extend(extend(intronic_regulated_gene_rn, immature_risc_rn), immature_translation_incorrect_rn)

    base_iFFL = extend(extend(CDS_intron_rn, common_rn), missplicing_rn)
    base_params = [:r_splicing => 2.0e-3, :r_mRNA_only => 0.0, :r_miR_only => 0.0]

    more_mRNA_params = [:r_splicing => 2.0e-3, :r_mRNA_only => 2.0e-3, :r_miR_only => 0.0]

    more_miR_params = [:r_splicing => 2.0e-3, :r_mRNA_only => 0.0, :r_miR_only => 2.0e-3]

end

In [ ]:
species(base_iFFL)

# Base iFFL simulations

In [ ]:
@everywhere risc_vals = [10000]
@everywhere moi_vals = [0.3, 3, 10]

for k=1:lastindex(risc_vals)
    #iterate through all initial values of risc
    for j=1:lastindex(moi_vals)
        #iterate through all moi values for Poisson distribution
        
        @everywhere begin
            #define copy number distribution
            CN_distribution = truncated(Poisson(moi_vals[$j]), lower=1.0)

            #define method to resample from copy number distribution
            function prob_func(prob, i, repeat)
                remake(prob, u0 = [:gene => rand(CN_distribution), :risc => risc_vals[$k], :immature_mRNA => 0, :control_mRNA => 0, :miRNA => 0, 
                    :regulated_mRNA => 0, :protein => 0, :control_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0,
                    :risc_miRNA_im_mRNA => 0, :incorrect_protein => 0])
            end

            #define simulation output function
            function output_func(sol, i)
                sol.u[end,:], false
            end

            #define initial values for simulation
            u0 = [:gene => rand(CN_distribution), :risc => risc_vals[$k], :immature_mRNA => 0, :control_mRNA => 0, :miRNA => 0, 
                    :regulated_mRNA => 0, :protein => 0, :control_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0,
                    :risc_miRNA_im_mRNA => 0, :incorrect_protein => 0]

            #define tspan
            tspan = (0., 100000.)
            
            #define jump problem for desired reaction network
            jprob = JumpProblem(base_iFFL, DiscreteProblem(base_iFFL, u0, tspan, base_params), SortingDirect(), save_positions=(false,false))

            #define ensemble problem with functions from above
            eprob = EnsembleProblem(jprob, prob_func=prob_func, output_func=output_func, safetycopy=false)

        end

        #solve given ensemble problem
        solns = solve(eprob, SSAStepper(), EnsembleSplitThreads(); trajectories=10000)

        #extract desired data from output
        reg_vec = zeros(10000)
        unreg_vec = zeros(10000)
        cn_vec = zeros(10000)
        for n = 1:lastindex(solns)
            reg_vec[n] = solns[n][1][4]
            unreg_vec[n] = solns[n][1][6]
            cn_vec[n] = solns[n][1][14]
        end

        df = DataFrame(copynum=cn_vec, reg_gene=reg_vec, unreg_gene=unreg_vec, design="Base", moi=moi_vals[j], risc=risc_vals[k])
        append!(data,df)

        f = Figure()
        ax1 = Axis(f[1,1],title="Base parameters: "*string(risc_vals[k])*" RISC, "*string(moi_vals[j])*" MOI", xlabel = "Copy Number", ylabel = "Protein Expression", limits = (nothing, nothing, 0, 10000)
            )
        s1 = scatter!(ax1, df.copynum, df.unreg_gene, color=(:gray, 0.5), label="unregulated gene")
        s2 = scatter!(ax1, df.copynum, df.reg_gene, color=(:blue, 0.5), label="regulated gene")
        axislegend(position=:lt)
        ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 10000))
        linkyaxes!(ax1,ax2)
        d1 = density!(df.unreg_gene, color=(:gray, 0.5), direction = :y)
        d2 = density!(df.reg_gene, color=(:blue, 0.5), direction = :y)
        xlims!(ax2, low = 0)
        hidedecorations!(ax2, grid = false)
        colsize!(f.layout, 1, Auto(3))
        save("$outdir/base_"*string(risc_vals[k])*"risc_"*string(moi_vals[j])*"moi.svg",f)
    end

end

In [ ]:
Parquet2.writefile("$outdir/stochastic_sims_base.gzip", data; compression_codec=:gzip)
Parquet2.writefile("$outdir/stochastic_sims.gzip", data; compression_codec=:gzip)

In [ ]:
data_base = copy(data)

# More mRNA simulations

In [ ]:
data_more_mRNA = DataFrame(copynum=Int[], reg_gene=Int[], unreg_gene=Int[], design=String[], moi=Float64[], risc=Int[])

In [ ]:
@everywhere risc_vals = [10000]
@everywhere moi_vals = [0.3, 3, 10]

for k=1:lastindex(risc_vals)
    #iterate through all initial values of risc
    for j=1:lastindex(moi_vals)
        #iterate through all moi values for Poisson distribution
        
        @everywhere begin
            #define copy number distribution
            CN_distribution = truncated(Poisson(moi_vals[$j]), lower=1.0)

            #define method to resample from copy number distribution
            function prob_func(prob, i, repeat)
                remake(prob, u0 = [:gene => rand(CN_distribution), :risc => risc_vals[$k], :immature_mRNA => 0, :control_mRNA => 0, :miRNA => 0, 
                    :regulated_mRNA => 0, :protein => 0, :control_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0,
                    :risc_miRNA_im_mRNA => 0, :incorrect_protein => 0])
            end

            #define simulation output function
            function output_func(sol, i)
                sol.u[end,:], false
            end

            #define initial values for simulation
            u0 = [:gene => rand(CN_distribution), :risc => risc_vals[$k], :immature_mRNA => 0, :control_mRNA => 0, :miRNA => 0, 
                    :regulated_mRNA => 0, :protein => 0, :control_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0,
                    :risc_miRNA_im_mRNA => 0, :incorrect_protein => 0]

            #define tspan
            tspan = (0., 100000.)
            
            #define jump problem for desired reaction network
            jprob = JumpProblem(base_iFFL, DiscreteProblem(base_iFFL, u0, tspan, more_mRNA_params), SortingDirect(), save_positions=(false,false))

            #define ensemble problem with functions from above
            eprob = EnsembleProblem(jprob, prob_func=prob_func, output_func=output_func, safetycopy=false)

        end

        #solve given ensemble problem
        solns = solve(eprob, SSAStepper(), EnsembleSplitThreads(); trajectories=10000)

        #extract desired data from output
        reg_vec = zeros(10000)
        unreg_vec = zeros(10000)
        cn_vec = zeros(10000)
        for n = 1:lastindex(solns)
            reg_vec[n] = solns[n][1][4]
            unreg_vec[n] = solns[n][1][6]
            cn_vec[n] = solns[n][1][14]
        end

        df = DataFrame(copynum=cn_vec, reg_gene=reg_vec, unreg_gene=unreg_vec, design="more mRNA", moi=moi_vals[j], risc=risc_vals[k])
        append!(data,df)
        append!(data_more_mRNA,df)

        f = Figure()
        ax1 = Axis(f[1,1],title="More mRNA: "*string(risc_vals[k])*" RISC, "*string(moi_vals[j])*" MOI", xlabel = "Copy Number", ylabel = "Protein Expression", limits = (nothing, nothing, 0, 10000)
            )
        s1 = scatter!(ax1, df.copynum, df.unreg_gene, color=(:gray, 0.5), label="unregulated gene")
        s2 = scatter!(ax1, df.copynum, df.reg_gene, color=(:blue, 0.5), label="regulated gene")
        axislegend(position=:lt)
        ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 10000))
        linkyaxes!(ax1,ax2)
        d1 = density!(df.unreg_gene, color=(:gray, 0.5), direction = :y)
        d2 = density!(df.reg_gene, color=(:blue, 0.5), direction = :y)
        xlims!(ax2, low = 0)
        hidedecorations!(ax2, grid = false)
        colsize!(f.layout, 1, Auto(3))
        save("$outdir/more_mRNA_"*string(risc_vals[k])*"risc_"*string(moi_vals[j])*"moi.svg",f)
    end

end

In [ ]:
Parquet2.writefile("$outdir/stochastic_sims.gzip", data; compression_codec=:gzip)
Parquet2.writefile("$outdir/stochastic_sims_more_mRNA.gzip", data_more_mRNA; compression_codec=:gzip)

# More miRNA simulations

In [ ]:
data_more_miR = DataFrame(copynum=Int[], reg_gene=Int[], unreg_gene=Int[], design=String[], moi=Float64[], risc=Int[])

In [ ]:
@everywhere risc_vals = [10000]
@everywhere moi_vals = [0.3, 3, 10]

for k=1:lastindex(risc_vals)
    #iterate through all initial values of risc
    for j=1:lastindex(moi_vals)
        #iterate through all moi values for Poisson distribution
        
        @everywhere begin
            #define copy number distribution
            CN_distribution = truncated(Poisson(moi_vals[$j]), lower=1.0)

            #define method to resample from copy number distribution
            function prob_func(prob, i, repeat)
                remake(prob, u0 = [:gene => rand(CN_distribution), :risc => risc_vals[$k], :immature_mRNA => 0, :control_mRNA => 0, :miRNA => 0, 
                    :regulated_mRNA => 0, :protein => 0, :control_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0,
                    :risc_miRNA_im_mRNA => 0, :incorrect_protein => 0])
            end

            #define simulation output function
            function output_func(sol, i)
                sol.u[end,:], false
            end

            #define initial values for simulation
            u0 = [:gene => rand(CN_distribution), :risc => risc_vals[$k], :immature_mRNA => 0, :control_mRNA => 0, :miRNA => 0, 
                    :regulated_mRNA => 0, :protein => 0, :control_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0,
                    :risc_miRNA_im_mRNA => 0, :incorrect_protein => 0]

            #define tspan
            tspan = (0., 100000.)
            
            #define jump problem for desired reaction network
            jprob = JumpProblem(base_iFFL, DiscreteProblem(base_iFFL, u0, tspan, more_miR_params), SortingDirect(), save_positions=(false,false))

            #define ensemble problem with functions from above
            eprob = EnsembleProblem(jprob, prob_func=prob_func, output_func=output_func, safetycopy=false)

        end

        #solve given ensemble problem
        solns = solve(eprob, SSAStepper(), EnsembleSplitThreads(); trajectories=10000)

        #extract desired data from output
        reg_vec = zeros(10000)
        unreg_vec = zeros(10000)
        cn_vec = zeros(10000)
        for n = 1:lastindex(solns)
            reg_vec[n] = solns[n][1][4]
            unreg_vec[n] = solns[n][1][6]
            cn_vec[n] = solns[n][1][14]
        end

        df = DataFrame(copynum=cn_vec, reg_gene=reg_vec, unreg_gene=unreg_vec, design="more miR", moi=moi_vals[j], risc=risc_vals[k])
        append!(data,df)
        append!(data_more_miR,df)

        f = Figure()
        ax1 = Axis(f[1,1],title="More miRNA: "*string(risc_vals[k])*" RISC, "*string(moi_vals[j])*" MOI", xlabel = "Copy Number", ylabel = "Protein Expression", limits = (nothing, nothing, 0, 10000)
            )
        s1 = scatter!(ax1, df.copynum, df.unreg_gene, color=(:gray, 0.5), label="unregulated gene")
        s2 = scatter!(ax1, df.copynum, df.reg_gene, color=(:blue, 0.5), label="regulated gene")
        axislegend(position=:lt)
        ax2 = Axis(f[1,2],limits = (nothing, nothing, 0, 10000))
        linkyaxes!(ax1,ax2)
        d1 = density!(df.unreg_gene, color=(:gray, 0.5), direction = :y)
        d2 = density!(df.reg_gene, color=(:blue, 0.5), direction = :y)
        xlims!(ax2, low = 0)
        hidedecorations!(ax2, grid = false)
        colsize!(f.layout, 1, Auto(3))
        save("$outdir/more_miRNA_"*string(risc_vals[k])*"risc_"*string(moi_vals[j])*"moi.svg",f)
    end

end

In [ ]:
Parquet2.writefile("$outdir/stochastic_sims.gzip", data; compression_codec=:gzip)
Parquet2.writefile("$outdir/stochastic_sims_more_miRNA.gzip", data_more_miR; compression_codec=:gzip)